In [1]:
!pip install -U cjio

You should consider upgrading via the '/Users/binyu/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip show cjio

Name: cjio
Version: 0.9.0
Summary: CLI to process and manipulate CityJSON files
Home-page: https://github.com/cityjson/cjio
Author: Hugo Ledoux, Balázs Dukai
Author-email: h.ledoux@tudelft.nl, balazs.dukai@3dgi.nl
License: MIT
Location: /Users/binyu/opt/anaconda3/lib/python3.9/site-packages
Requires: Click, numpy
Required-by: 


In [5]:
import pandas as pd
from pathlib import Path
from copy import deepcopy
from cjio import cityjson
from cjio.models import CityObject, Geometry
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
plt.close('all')
from sklearn.preprocessing import FunctionTransformer
from sklearn import cluster
import numpy as np

In [39]:
am_perception = pd.read_csv('/Users/binyu/Downloads/amsterdam_all_cluster_0110.csv')
am_perception.head()

,Unnamed: 0,Unnamed: 0.1,id,type,lod,point_count,unique_point_count,surface_count,actual_volume,convex_hull_volume,...,statnaam,identificatie,WKT,originality,pleasing,ordered,boring,complexity,morphology_cluster,perception_cluster
0,0,1495,NL.IMBAG.Pand.0363100012164990-0,BuildingPart,2.2,378,122,69,1268.156874,1402.025853,...,Amsterdam,NL.IMBAG.Pand.0363100012164990,"POLYGON ((120853.348440981 487282.656825949,12...",4.580033,5.111876,3.765070,4.686961,5.177428,0,0_0
1,1,1496,NL.IMBAG.Pand.0363100012164992-0,BuildingPart,2.2,2842,927,508,23609.009330,29906.687950,...,Amsterdam,NL.IMBAG.Pand.0363100012164992,"POLYGON ((120888.309340922 487342.969326587,12...",4.436723,4.750063,3.537349,4.830935,4.790968,0,0_0
2,2,1496,NL.IMBAG.Pand.0363100012164992-0,BuildingPart,2.2,2842,927,508,23609.009330,29906.687950,...,Amsterdam,NL.IMBAG.Pand.0363100012164992,"POLYGON ((120877.637440872 487337.031826483,12...",4.680998,4.956201,3.564067,5.079813,5.017166,0,0_0
3,3,1500,NL.IMBAG.Pand.0363100012164995-0,BuildingPart,2.2,718,231,130,3897.663603,5393.516100,...,Amsterdam,NL.IMBAG.Pand.0363100012164995,"POLYGON ((120637.07493809 487505.219326411,120...",4.320895,4.844801,3.979450,4.560967,5.058178,0,0_0
4,4,1501,NL.IMBAG.Pand.0363100012164996-0,BuildingPart,2.2,338,108,63,1162.978646,1516.772896,...,Amsterdam,NL.IMBAG.Pand.0363100012164996,"POLYGON ((120707.746838933 487451.188026405,12...",4.262244,4.748159,3.847853,5.201878,5.374521,0,0_0


In [40]:
originality = pd.Series(am_perception.originality.values, index=am_perception.identificatie).to_dict()
pleasing = pd.Series(am_perception.pleasing.values, index=am_perception.identificatie).to_dict()
ordered = pd.Series(am_perception.ordered.values, index=am_perception.identificatie).to_dict()
boring = pd.Series(am_perception.boring.values, index=am_perception.identificatie).to_dict()
complexity = pd.Series(am_perception.complexity.values, index=am_perception.identificatie).to_dict()


def add_originality(co_id):
    return originality.get(co_id)

def add_pleasing(co_id):
    return pleasing.get(co_id)

def add_ordered(co_id):
    return ordered.get(co_id)

def add_boring(co_id):
    return boring.get(co_id)

def add_complexity(co_id):
    return complexity.get(co_id)


In [4]:
import json
import pandas as pd
import os



# Path to the folder containing CityJSON files
folder_path = '/Users/binyu/Downloads/tiles/upgrade'
output_folder_path = '/Users/binyu/Downloads/tiles/'

# Ensure output folder exists
os.makedirs(output_folder_path, exist_ok=True)

# Iterate over each CityJSON file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.city.json'):
        file_path = os.path.join(folder_path, filename)
        
        # Load the CityJSON file
        with open(file_path, 'r') as file:
            cityjson_data = json.load(file)
        
        # Add the Extension    
        cityjson_data["extensions"] = {
            "Perception": {
                "url": "https://raw.githubusercontent.com/binyulei/extension/main/perception.ext.json",
                "version": "0.5"
            }
        }
        
        # Modify the cityobjects
        for co_id, co in cityjson_data["CityObjects"].items():
            co["attributes"]["morphology_cluster"] = add_morphology_cluster(co_id)
            co["attributes"]['perception_cluster'] = add_perception_cluster(co_id)
            if "+perception-originality" not in co["attributes"]:
                co["attributes"]["+perception-originality"] = {}
            if "+perception-pleasing" not in co["attributes"]:
                co["attributes"]["+perception-pleasing"] = {}
            if "+perception-ordered" not in co["attributes"]:
                co["attributes"]["+perception-ordered"] = {}
            if "+perception-boring" not in co["attributes"]:
                co["attributes"]["+perception-boring"] = {}
            if "+perception-complexity" not in co["attributes"]:
                co["attributes"]["+perception-complexity"] = {}
                
            # Add the perception attributes
            co["attributes"]["+perception-originality"]["originality"] = add_originality(co_id)
            co["attributes"]["+perception-pleasing"]["pleasing"] = add_pleasing(co_id)
            co["attributes"]["+perception-ordered"]["ordered"] = add_ordered(co_id)
            co["attributes"]["+perception-boring"]["boring"] = add_boring(co_id)
            co["attributes"]["+perception-complexity"]["complexity"] = add_complexity(co_id)

        # Save the modified CityJSON file
        output_file_path = os.path.join(output_folder_path, f'0117_{filename}')
        with open(output_file_path, 'w') as file:
            json.dump(cityjson_data, file, indent=4)

        print(f"Processed and saved: {filename}")

Processed and saved: add_10-426-722_new.city.json
Processed and saved: add_10-430-720_new.city.json
Processed and saved: add_10-424-722_new.city.json
Processed and saved: add_10-430-724_new.city.json
Processed and saved: add_9-424-724_new.city.json
Processed and saved: add_10-428-724_new.city.json
Processed and saved: add_10-428-720_new.city.json
Processed and saved: add_10-430-726_new.city.json
Processed and saved: add_10-424-720_new.city.json
Processed and saved: add_10-430-722_new.city.json
Processed and saved: add_10-428-722_new.city.json
Processed and saved: add_10-428-726_new.city.json
Processed and saved: add_10-426-720_new.city.json
